In [ ]:
Hard

In [ ]:
Задание 1

In [1]:
from sympy import symbols, groebner, reduced
from sympy.polys.polytools import Poly

def is_polynomial_in_ideal(f_expr, generators, variables):
    """
    Проверяет, принадлежит ли полином f идеалу, порожденному generators.
    
    Args:
        f_expr: целевой полином (выражение sympy).
        generators: список полиномов, порождающих идеал S.
        variables: кортеж переменных (x, y, z...).
        
    Returns:
        bool: True, если f принадлежит идеалу, иначе False.
        expr: Остаток от деления (для наглядности).
    """
    
    G = groebner(generators, *variables, order='lex')
    
    rem = reduced(f_expr, G, *variables, order='lex')[1]
    
    return rem == 0, rem

x, y, z = symbols('x y z')

generators1 = [x + y, x - y]
f1 = x

in_ideal, remainder = is_polynomial_in_ideal(f1, generators1, (x, y))
print(f"Пример 1:")
print(f"Идеал: {generators1}")
print(f"Полином: {f1}")
print(f"Принадлежит идеалу? -> {in_ideal}")
print(f"Остаток: {remainder}")

generators2 = [x**2 + y**2 - 1, y - x**2]
f2 = y**2 + y - 1 

in_ideal, remainder = is_polynomial_in_ideal(f2, generators2, (x, y))
print(f"Пример 2:")
print(f"Идеал: {generators2}")
print(f"Полином: {f2}")
print(f"Принадлежит идеалу? -> {in_ideal}") # Ожидаем True

generators3 = [x**2, y**2]
f3 = x * y 

in_ideal, remainder = is_polynomial_in_ideal(f3, generators3, (x, y))
print(f"Пример 3:")
print(f"Идеал: {generators3}")
print(f"Полином: {f3}")
print(f"Принадлежит идеалу? -> {in_ideal}") # Ожидаем False
print(f"Остаток: {remainder}")

Пример 1:
Идеал: [x + y, x - y]
Полином: x
Принадлежит идеалу? -> True
Остаток: 0
Пример 2:
Идеал: [x**2 + y**2 - 1, -x**2 + y]
Полином: y**2 + y - 1
Принадлежит идеалу? -> True
Пример 3:
Идеал: [x**2, y**2]
Полином: x*y
Принадлежит идеалу? -> False
Остаток: x*y


In [ ]:
Задание 2

In [2]:
import math
from functools import reduce
from sympy import symbols, gcd, Poly, RR, QQ

def solve_ideal_Z(numbers: list) -> int:
    """
    Решает задачу для кольца целых чисел Z.
    Идеал I = (a1, ..., ak).
    """
    print(f"--- Кольцо Z ---")
    print(f"Входные числа: {numbers}")
    
    is_principal = True
    print(f"Является ли I главным? -> {is_principal} (т.к. Z - кольцо главных идеалов)")
    
    d = reduce(math.gcd, numbers)
    
    print(f"Порождающий элемент d (НОД): {d}")
    print(f"Идеал I = ({d})")
    return d

def solve_ideal_Kx(polys_exprs: list):
    """
    Решает задачу для кольца полиномов K[x].
    Идеал I = (f1, ..., fk).
    """
    print(f"--- Кольцо K[x] ---")
    x = symbols('x')
    print(f"Входные полиномы: {polys_exprs}")
    
    is_principal = True
    print(f"Является ли I главным? -> {is_principal} (т.к. K[x] - кольцо главных идеалов)")
    
    polys = [Poly(p, x) for p in polys_exprs]
    
    d_poly = reduce(gcd, polys)
    
    print(f"Порождающий полином d(x) (НОД): {d_poly.as_expr()}")
    print(f"Идеал I = ({d_poly.as_expr()})")
    return d_poly


if __name__ == "__main__":
    x = symbols('x')
    nums = [30, 42, 72]
    solve_ideal_Z(nums)
    
    polynomials = [
        x**2 - 1,
        x**2 + 2*x + 1
    ]
    solve_ideal_Kx(polynomials)

--- Кольцо Z ---
Входные числа: [30, 42, 72]
Является ли I главным? -> True (т.к. Z - кольцо главных идеалов)
Порождающий элемент d (НОД): 6
Идеал I = (6)
--- Кольцо K[x] ---
Входные полиномы: [x**2 - 1, x**2 + 2*x + 1]
Является ли I главным? -> True (т.к. K[x] - кольцо главных идеалов)
Порождающий полином d(x) (НОД): x + 1
Идеал I = (x + 1)


In [ ]:
Задание 3

In [4]:
from typing import Union, List
import math

class RingElement:
    """Базовый класс для элементов колец: целых чисел и полиномов."""
    def __init__(self, data: Union[int, List[float]]):
        """
        data:
          - если это целое число -> элемент кольца Z;
          - если это список коэффициентов [a0, a1, ..., an] -> полином a0 + a1*x + ... + an*x^n.
        """
        if isinstance(data, list):
            while len(data) > 1 and abs(data[-1]) < 1e-9:
                data.pop()
            if not data: 
                data = [0]
        
        self.data = data
        self.is_polynomial = isinstance(data, list)

    def __repr__(self) -> str:
        if self.is_polynomial:
            if len(self.data) == 1 and self.data[0] == 0:
                return "0"
            terms = []
            for i, c in enumerate(self.data):
                if abs(c) > 1e-9: 
                    c_val = int(c) if c.is_integer() else c
                    if i == 0:
                        terms.append(f"{c_val}")
                    else:
                        terms.append(f"{c_val}*x^{i}")
            return " + ".join(terms) if terms else "0"
        return str(self.data)

    def is_zero(self) -> bool:
        if self.is_polynomial:
            return len(self.data) == 1 and abs(self.data[0]) < 1e-9
        return self.data == 0

    def degree(self) -> int:
        if not self.is_polynomial or self.is_zero():
            return 0
        return len(self.data) - 1

    def make_monic(self):
        """Делает полином приведенным (старший коэффициент = 1). Для чисел возвращает модуль."""
        if self.is_zero():
            return RingElement([0] if self.is_polynomial else 0)
        
        if self.is_polynomial:
            lc = self.data[-1]
            new_coeffs = [c / lc for c in self.data]
            return RingElement(new_coeffs)
        else:
            return RingElement(abs(self.data))

    def __eq__(self, other):
        return self.data == other.data

    def __mod__(self, other: 'RingElement') -> 'RingElement':
        """Реализация деления с остатком (%) для чисел и полиномов."""
        
        if not self.is_polynomial and not other.is_polynomial:
            return RingElement(self.data % other.data)

        if self.is_polynomial and other.is_polynomial:
            dividend = self.data[:] 
            divisor = other.data[:]

            while len(dividend) >= len(divisor) and not (len(dividend) == 1 and abs(dividend[0]) < 1e-9):
                deg_diff = len(dividend) - len(divisor)
                
                factor = dividend[-1] / divisor[-1]
                
                for i in range(len(divisor)):
                    dividend[i + deg_diff] -= divisor[i] * factor
                
                while len(dividend) > 0 and abs(dividend[-1]) < 1e-9:
                    dividend.pop()
                if not dividend:
                    dividend = [0]

            return RingElement(dividend)



def gcd_ring_elements(elements: List[RingElement]) -> RingElement:
    """
    Возвращает порождающий главного идеала (НОД элементов).
    """
    
    def gcd_two(a: RingElement, b: RingElement) -> RingElement:
        while not b.is_zero():
            a, b = b, a % b
        return a

    result = elements[0]
    for i in range(1, len(elements)):
        result = gcd_two(result, elements[i])
    return result.make_monic()

print("--- Тест Z (Целые числа) ---")
z1 = RingElement(30)
z2 = RingElement(42)
z3 = RingElement(72)
gcd_z = gcd_ring_elements([z1, z2, z3])
print(f"GCD({z1}, {z2}, {z3}) = {gcd_z}")


print("\n--- Тест K[x] (Полиномы) ---")

p1 = RingElement([-1.0, 0.0, 1.0])
p2 = RingElement([1.0, 2.0, 1.0])

gcd_p = gcd_ring_elements([p1, p2])
print(f"P1 = {p1}")
print(f"P2 = {p2}")
print(f"GCD(P1, P2) = {gcd_p}")

--- Тест Z (Целые числа) ---
GCD(30, 42, 72) = 6

--- Тест K[x] (Полиномы) ---
P1 = -1 + 1*x^2
P2 = 1 + 2*x^1 + 1*x^2
GCD(P1, P2) = 1 + 1*x^1
